## Get Started with Spark NLP for Healthcare

## Getting the keys and installation

1. In order to get trial keys for Spark NLP for Healthcare
, fill the form at https://www.johnsnowlabs.com/spark-nlp-try-free/ and you will get your keys to your email in a few minutes.

2. On a new cluster or existing one

  - add the following to the `Advanced Options -> Spark` tab, in `Spark.Config` box:

    ```bash
    spark.local.dir /var
    spark.kryoserializer.buffer.max 1000M
    spark.serializer org.apache.spark.serializer.KryoSerializer
    ```
  - add the following to the `Advanced Options -> Spark` tab, in `Environment Variables` box:

    ```bash
    AWS_ACCESS_KEY_ID=xxx
    AWS_SECRET_ACCESS_KEY=yyy
    SPARK_NLP_LICENSE=zzz
    ```

3. Download the followings with AWS CLI to your local computer

    `$ aws s3 cp --region us-east-2 s3://pypi.johnsnowlabs.com/$jsl_secret/spark-nlp-jsl-$jsl_version.jar spark-nlp-jsl-$jsl_version.jar`

    `$ aws s3 cp --region us-east-2 s3://pypi.johnsnowlabs.com/$jsl_secret/spark-nlp-jsl/spark_nlp_jsl-$jsl_version-py3-none-any.whl spark_nlp_jsl-$jsl_version-py3-none-any.whl` 

4. In `Libraries` tab inside your cluster:

 - Install New -> PyPI -> `spark-nlp==$public_version` -> Install
 - Install New -> Maven -> Coordinates -> `com.johnsnowlabs.nlp:spark-nlp_2.12:$public_version` -> Install

 - add following jars for the Healthcare library that you downloaded above:
        - Install New -> Python Whl -> upload `spark_nlp_jsl-$jsl_version-py3-none-any.whl`

        - Install New -> Jar -> upload `spark-nlp-jsl-$jsl_version.jar`

5. Now you can attach your notebook to the cluster and use Spark NLP!

For more information, see 

  https://nlp.johnsnowlabs.com/docs/en/install#databricks-support

  https://nlp.johnsnowlabs.com/docs/en/licensed_install#install-spark-nlp-for-healthcare-on-databricks
  
The follwing notebook is prepared and tested on **r2.2xlarge at 8.0 (includes Apache Spark 3.1.1, Scala 2.12)** on Databricks

In order to get more detailed examples, please check this repository : https://github.com/JohnSnowLabs/spark-nlp-workshop/tree/master/tutorials/Certification_Trainings/Healthcare/databricks_notebooks

Let's import the libraries which we will use in the following cells.

In [0]:
import os
import json
import string
import numpy as np
import pandas as pd

import sparknlp
import sparknlp_jsl
from sparknlp.base import *
from sparknlp.util import *
from sparknlp.annotator import *
from sparknlp_jsl.base import *
from sparknlp_jsl.annotator import *
from sparknlp.pretrained import ResourceDownloader

from pyspark.sql import functions as F
from pyspark.ml import Pipeline, PipelineModel
from sparknlp.training import CoNLL

pd.set_option('max_colwidth', 100)
pd.set_option('display.max_columns', 100)  
pd.set_option('display.expand_frame_repr', False)

print('sparknlp.version : ',sparknlp.version())
print('sparknlp_jsl.version : ',sparknlp_jsl.version())

spark


sparknlp.version : 3.4.0
sparknlp_jsl.version : 3.4.0
Out[1]:

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v3.2.0 
 Master 
 spark://10.139.64.7:7077 
 AppName 
 Databricks Shell

**Read Dataset**

We will download a sample file and create a spark dataframe.

In [0]:
! wget -q https://s3.amazonaws.com/auxdata.johnsnowlabs.com/public/resources/en/pubmed/pubmed_sample_text_small.csv

In [0]:
pubMedDF = spark.read.option("header", "true").csv("dbfs:/pubmed_sample_text_small.csv")
                
pubMedDF.show(truncate=100)

+----------------------------------------------------------------------------------------------------+
 text|
+----------------------------------------------------------------------------------------------------+
The human KCNJ9 (Kir 3.3, GIRK3) is a member of the G-protein-activated inwardly rectifying potas...|
BACKGROUND: At present, it is one of the most important issues for the treatment of breast cancer...|
OBJECTIVE: To investigate the relationship between preoperative atrialfibrillation and early and ...|
Combined EEG/fMRI recording has been used to localize the generators of EEGevents and to identify...|
Kohlschutter syndrome is a rare neurodegenerative disorder presenting withintractable seizures, d...|
Statistical analysis of neuroimages is commonly approached with intergroupcomparisons made by rep...|
The synthetic DOX-LNA conjugate was characterized by proton nuclear magneticresonance and mass sp...|
Our objective was to compare three different methods of blood pressuremeasurement through the res...|
We conducted a phase II study to assess the efficacy and tolerability ofirinotecan and cisplatin ...|
"""Monomeric sarcosine oxidase (MSOX) is a flavoenzyme that catalyzes the oxidative demethylation...|
We presented the tachinid fly Exorista japonica with moving host models: afreeze-dried larva of t...|
The literature dealing with the water conducting properties of sapwood xylem intrees is inconsist...|
A novel approach to synthesize chitosan-O-isopropyl-5'-O-d4T monophosphateconjugate was developed...|
An HPLC-ESI-MS-MS method has been developed for the quantitative determination offour diterpenoid...|
The localizing and lateralizing values of eye and head ictal deviations duringfrontal lobe seizur...|
OBJECTIVE: To evaluate the effectiveness and acceptability of expectantmanagement of induced and ...|
For the construction of new combinatorial libraries, a lead compound was created by replacing the...|
We report the results of a screen for genetic association with urinary arsenicmetabolite levels i...|
Intraparenchymal pericatheter cyst is rarely reported. Obstruction in theventriculoperitoneal shu...|
It is known that patients with Klinefelter's syndrome are inclined to developconcomitant malignan...|
+----------------------------------------------------------------------------------------------------+
only showing top 20 rows

## 1. Clinical NER Pipeline
We will extract clinical entities from text by using `ner_clinical_large` model.

In [0]:
# Annotator that transforms a text column from dataframe into an Annotation ready for NLP
documentAssembler = DocumentAssembler()\
        .setInputCol("text")\
        .setOutputCol("document")

# Sentence Detector annotator, processes various sentences per line

#sentenceDetector = SentenceDetector()\
        #.setInputCols(["document"])\
        #.setOutputCol("sentence")
sentenceDetector = SentenceDetectorDLModel.pretrained("sentence_detector_dl_healthcare","en","clinical/models")\
        .setInputCols(["document"])\
        .setOutputCol("sentence")
 
# Tokenizer splits words in a relevant format for NLP
tokenizer = Tokenizer()\
        .setInputCols(["sentence"])\
        .setOutputCol("token")

# Clinical word embeddings trained on PubMED dataset
word_embeddings = WordEmbeddingsModel.pretrained("embeddings_clinical","en","clinical/models")\
        .setInputCols(["sentence","token"])\
        .setOutputCol("embeddings")

pos_tagger = PerceptronModel.pretrained("pos_clinical", "en", "clinical/models") \
        .setInputCols(["sentence", "token"])\
        .setOutputCol("pos_tags")

# NER model trained on i2b2 (sampled from MIMIC) dataset
clinical_ner = MedicalNerModel.pretrained("ner_clinical_large","en","clinical/models")\
        .setInputCols(["sentence","token","embeddings"])\
        .setOutputCol("ner")

ner_converter = NerConverter()\
        .setInputCols(["sentence","token","ner"])\
        .setOutputCol("ner_chunk")

nerPipeline = Pipeline(stages=[
        documentAssembler,
        sentenceDetector,
        tokenizer,
        word_embeddings,
        pos_tagger,
        clinical_ner,
        ner_converter])


empty_data = spark.createDataFrame([[""]]).toDF("text")

ner_model = nerPipeline.fit(empty_data)

sentence_detector_dl_healthcare download started this may take some time.
Approximate size to download 367.3 KB
[ | ][ / ][ — ][OK!]
embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][OK!]
pos_clinical download started this may take some time.
Approximate size to download 1.5 MB
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][OK!]
ner_clinical_large download started this may take some time.
Approximate size to download 13.9 MB
[ | ][ / ][OK!]

In [0]:
ner_model.stages

Out[4]: [DocumentAssembler_dbb96eaca92c,
 SentenceDetectorDLModel_6bafc4746ea5,
 REGEX_TOKENIZER_af910a2bc3ab,
 WORD_EMBEDDINGS_MODEL_9004b1d00302,
 POS_6f55785005bf,
 MedicalNerModel_1a8637089929,
 NerConverter_4f3c61f53b58]

In [0]:
clinical_ner.getClasses()

Out[5]: ['O',
 'B-TREATMENT',
 'I-TREATMENT',
 'B-PROBLEM',
 'I-PROBLEM',
 'B-TEST',
 'I-TEST']

In [0]:
result = ner_model.transform(pubMedDF.limit(100))

In [0]:
result.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
 text| document| sentence| token| embeddings| pos_tags| ner| ner_chunk|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
A numerical metho...|[{document, 0, 88...|[{document, 0, 15...|[{token, 0, 0, A,...|[{word_embeddings...|[{pos, 0, 0, DD, ...|[{named_entity, 0...|[{chunk, 69, 95, ...|
Recording of the ...|[{document, 0, 44...|[{document, 0, 10...|[{token, 0, 8, Re...|[{word_embeddings...|[{pos, 0, 8, VVGN...|[{named_entity, 0...|[{chunk, 13, 30, ...|
Colicin E5 cleave...|[{document, 0, 12...|[{document, 0, 11...|[{token, 0, 6, Co...|[{word_embeddings...|[{pos, 0, 6, NN, ...|[{named_entity, 0...|[{chunk, 0, 9, Co...|
Recombinant adeno...|[{document, 0, 13...|[{document, 0, 20...|[{token, 0, 10, R...|[{word_embeddings...|[{pos, 0, 10, JJ,...|[{named_entity, 0...|[{chunk, 0, 28, R...|
To understand its...|[{document, 0, 29...|[{document, 0, 19...|[{token, 0, 1, To...|[{word_embeddings...|[{pos, 0, 1, TO, ...|[{named_entity, 0...|[{chunk, 37, 52, ...|
OBJECTIVE: To rev...|[{document, 0, 20...|[{document, 0, 20...|[{token, 0, 8, OB...|[{word_embeddings...|[{pos, 0, 8, NN, ...|[{named_entity, 0...|[{chunk, 63, 91, ...|
AIM: Understandin...|[{document, 0, 15...|[{document, 0, 16...|[{token, 0, 2, AI...|[{word_embeddings...|[{pos, 0, 2, NN, ...|[{named_entity, 0...|[{chunk, 46, 74, ...|
In the present st...|[{document, 0, 92...|[{document, 0, 21...|[{token, 0, 1, In...|[{word_embeddings...|[{pos, 0, 1, II, ...|[{named_entity, 0...|[{chunk, 3, 19, t...|
AIM: To describe ...|[{document, 0, 14...|[{document, 0, 27...|[{token, 0, 2, AI...|[{word_embeddings...|[{pos, 0, 2, NN, ...|[{named_entity, 0...|[{chunk, 17, 32, ...|
Although the risk...|[{document, 0, 12...|[{document, 0, 11...|[{token, 0, 7, Al...|[{word_embeddings...|[{pos, 0, 7, CS, ...|[{named_entity, 0...|[{chunk, 118, 163...|
Successful long-t...|[{document, 0, 16...|[{document, 0, 12...|[{token, 0, 9, Su...|[{word_embeddings...|[{pos, 0, 9, JJ, ...|[{named_entity, 0...|[{chunk, 0, 43, S...|
Pulmonary alveola...|[{document, 0, 16...|[{document, 0, 15...|[{token, 0, 8, Pu...|[{word_embeddings...|[{pos, 0, 8, JJ, ...|[{named_entity, 0...|[{chunk, 68, 94, ...|
Prenatal care is ...|[{document, 0, 26...|[{document, 0, 68...|[{token, 0, 7, Pr...|[{word_embeddings...|[{pos, 0, 7, JJ, ...|[{named_entity, 0...|[{chunk, 70, 86, ...|
Controversy exist...|[{document, 0, 12...|[{document, 0, 18...|[{token, 0, 10, C...|[{word_embeddings...|[{pos, 0, 10, NN,...|[{named_entity, 0...|[{chunk, 43, 66, ...|
Children with a f...|[{document, 0, 34...|[{document, 0, 13...|[{token, 0, 7, Ch...|[{word_embeddings...|[{pos, 0, 7, NNS,...|[{named_entity, 0...|[{chunk, 34, 46, ...|
The mucosal immun...|[{document, 0, 19...|[{document, 0, 12...|[{token, 0, 2, Th...|[{word_embeddings...|[{pos, 0, 2, DD, ...|[{named_entity, 0...|[{chunk, 51, 57, ...|
Urinary concretio...|[{document, 0, 86...|[{document, 0, 15...|[{token, 0, 6, Ur...|[{word_embeddings...|[{pos, 0, 6, JJ, ...|[{named_entity, 0...|[{chunk, 0, 18, U...|
In this study, th...|[{document, 0, 14...|[{document, 0, 17...|[{token, 0, 1, In...|[{word_embeddings...|[{pos, 0, 1, II, ...|[{named_entity, 0...|[{chunk, 3, 12, t...|
The presence of c...|[{document, 0, 44...|[{document, 0, 12...|[{token, 0, 2, Th...|[{word_embeddings...|[{pos, 0, 2, DD, ...|[{named_entity, 0...|[{chunk, 16, 32, ...|
Phosphatidylethan...|[{document, 0, 17...|[{document, 0, 26...|[{token, 0, 23, P...|[{word_embeddings...|[{pos, 0, 23, NN,...|[{named_entity, 0...|[{chunk, 0, 48, P...|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
only showing top 20 rows

In [0]:
result.select("sentence.result").show(truncate=100)

+----------------------------------------------------------------------------------------------------+
 result|
+----------------------------------------------------------------------------------------------------+
[The human KCNJ9 (Kir 3.3, GIRK3) is a member of the G-protein-activated inwardly rectifying pota...|
[BACKGROUND: At present, it is one of the most important issues for the treatment of breast cance...|
[OBJECTIVE: To investigate the relationship between preoperative atrialfibrillation and early and...|
[Combined EEG/fMRI recording has been used to localize the generators of EEGevents and to identif...|
[Kohlschutter syndrome is a rare neurodegenerative disorder presenting withintractable seizures, ...|
[Statistical analysis of neuroimages is commonly approached with intergroupcomparisons made by re...|
[The synthetic DOX-LNA conjugate was characterized by proton nuclear magneticresonance and mass s...|
[Our objective was to compare three different methods of blood pressuremeasurement through the re...|
[We conducted a phase II study to assess the efficacy and tolerability ofirinotecan and cisplatin...|
["""Monomeric sarcosine oxidase (MSOX) is a flavoenzyme that catalyzes the oxidative demethylatio...|
[We presented the tachinid fly Exorista japonica with moving host models: afreeze-dried larva of ...|
[The literature dealing with the water conducting properties of sapwood xylem intrees is inconsis...|
[A novel approach to synthesize chitosan-O-isopropyl-5'-O-d4T monophosphateconjugate was develope...|
[An HPLC-ESI-MS-MS method has been developed for the quantitative determination offour diterpenoi...|
[The localizing and lateralizing values of eye and head ictal deviations duringfrontal lobe seizu...|
[OBJECTIVE: To evaluate the effectiveness and acceptability of expectantmanagement of induced and...|
[For the construction of new combinatorial libraries, a lead compound was created by replacing th...|
[We report the results of a screen for genetic association with urinary arsenicmetabolite levels ...|
[Intraparenchymal pericatheter cyst is rarely reported., Obstruction in theventriculoperitoneal s...|
[It is known that patients with Klinefelter's syndrome are inclined to developconcomitant maligna...|
+----------------------------------------------------------------------------------------------------+
only showing top 20 rows

In [0]:
from pyspark.sql import functions as F

result_df = result.select(F.explode(F.arrays_zip(result.token.result,result.ner.result)).alias("cols"))\
                  .select(F.expr("cols['0']").alias("token"),
                          F.expr("cols['1']").alias("ner_label"))

result_df.show(50, truncate=100)

+-------------------+-----------+
 token| ner_label|
+-------------------+-----------+
 The| O|
 human| O|
 KCNJ9| O|
 (| O|
 Kir| O|
 3.3| O|
 ,| O|
 GIRK3| O|
 )| O|
 is| O|
 a| O|
 member| O|
 of| O|
 the|B-TREATMENT|
G-protein-activated|I-TREATMENT|
 inwardly|I-TREATMENT|
 rectifying|I-TREATMENT|
 potassium|I-TREATMENT|
 (|I-TREATMENT|
 GIRK|I-TREATMENT|
 )| O|
 channel| O|
 family| O|
 .| O|
 Here| O|
 we| O|
 describe| O|
 the|B-TREATMENT|
genomicorganization|I-TREATMENT|
 of| O|
 the| O|
 KCNJ9| O|
 locus| O|
 on| O|
 chromosome| O|
 1q21-23| O|
 as| O|
 a| B-PROBLEM|
 candidate| I-PROBLEM|
 gene| I-PROBLEM|
 forType| I-PROBLEM|
 II| I-PROBLEM|
 diabetes| I-PROBLEM|
 mellitus| I-PROBLEM|
 in| O|
 the| O|
 Pima| O|
 Indian| O|
 population| O|
 .| O|
+-------------------+-----------+
only showing top 50 rows

Lets count the ner_labels.

In [0]:
result_df.select("token", "ner_label").groupBy('ner_label').count().orderBy('count', ascending=False).show(truncate=False)

+-----------+-----+
ner_label |count|
+-----------+-----+
O |12587|
I-PROBLEM |1737 |
B-PROBLEM |1008 |
I-TREATMENT|789 |
B-TREATMENT|655 |
I-TEST |573 |
B-TEST |457 |
+-----------+-----+

In [0]:
result.select(F.explode(F.arrays_zip(result.ner_chunk.result, result.ner_chunk.begin, result.ner_chunk.end, result.ner_chunk.metadata)).alias("cols")) \
      .select(F.expr("cols['3']['sentence']").alias("sentence_id"),
              F.expr("cols['0']").alias("chunk"),
              F.expr("cols['1']").alias("begin"),
              F.expr("cols['2']").alias("end"),
              F.expr("cols['3']['entity']").alias("ner_label")).show(truncate=False)

+-----------+-------------------------------------------------+-----+---+---------+
sentence_id|chunk |begin|end|ner_label|
+-----------+-------------------------------------------------+-----+---+---------+
0 |far-fieldoptical scattering |69 |95 |PROBLEM |
0 |particles |102 |110|PROBLEM |
3 |themethod |380 |388|TREATMENT|
3 |a spherical particle |415 |434|PROBLEM |
3 |toMie scattering theory |462 |484|TREATMENT|
5 |small-amplitude oscillations |576 |603|PROBLEM |
5 |the numerical method |653 |672|TREATMENT|
6 |Numerical data fromspheroidal particles |675 |713|PROBLEM |
6 |hemispherical particles |719 |741|PROBLEM |
7 |ofhemispherical particles |771 |795|PROBLEM |
7 |a calibration |800 |812|TEST |
7 |intensity-type opticalparticle-sizing instruments|827 |875|TREATMENT|
0 |the esophageal ECG |13 |30 |TEST |
0 |tachycardia |90 |100|PROBLEM |
1 |A method |103 |110|TREATMENT|
1 |the Arzco pill electrode canbe |134 |163|TREATMENT|
2 |The electrode |174 |186|TREATMENT|
2 |atelemetric monitor |236 |254|TEST |
3 |the esophageal ECG |269 |286|TEST |
3 |transient, self-limiting tachycardias |343 |379|PROBLEM |
+-----------+-------------------------------------------------+-----+---+---------+
only showing top 20 rows

**We can also filter NER results to get specific entities by using `setWhiteList()` parameter. In this example we will get only `PROBLEM` entities.**

In [0]:
ner_converter_filter = NerConverter()\
        .setInputCols(["sentence","token","ner"])\
        .setOutputCol("ner_chunk")\
        .setWhiteList(["PROBLEM"])

nerFilteredPipeline = Pipeline(stages=[
        documentAssembler,
        sentenceDetector,
        tokenizer,
        word_embeddings,
        clinical_ner,
        ner_converter_filter])


empty_data = spark.createDataFrame([[""]]).toDF("text")

ner_filtered_model = nerFilteredPipeline.fit(empty_data)

In [0]:
filtered_result = ner_filtered_model.transform(pubMedDF.limit(100))

In [0]:
filtered_result.select(F.explode(F.arrays_zip(filtered_result.ner_chunk.result, 
                                              filtered_result.ner_chunk.begin, 
                                              filtered_result.ner_chunk.end, 
                                              filtered_result.ner_chunk.metadata)).alias("cols")) \
               .select(F.expr("cols['3']['sentence']").alias("sentence_id"),
                       F.expr("cols['0']").alias("chunk"),
                       F.expr("cols['1']").alias("begin"),
                       F.expr("cols['2']").alias("end"),
                       F.expr("cols['3']['entity']").alias("ner_label")).show(truncate=False)

+-----------+---------------------------------------------+-----+----+---------+
sentence_id|chunk |begin|end |ner_label|
+-----------+---------------------------------------------+-----+----+---------+
1 |a candidate gene forType II diabetes mellitus|210 |254 |PROBLEM |
3 |aVal366Ala substitution |532 |554 |PROBLEM |
3 |an 8 base-pair |561 |574 |PROBLEM |
3 |insertion/deletion |581 |598 |PROBLEM |
4 |the transcript in various humantissues |648 |685 |PROBLEM |
4 |fat andskeletal muscle |749 |770 |PROBLEM |
5 |furtherstudies |830 |843 |PROBLEM |
5 |Type II diabetes |940 |955 |PROBLEM |
0 |breast cancer |84 |96 |PROBLEM |
6 |change |866 |871 |PROBLEM |
9 |Themajor toxicity |1110 |1126|PROBLEM |
9 |myelosuppression |1132 |1147|PROBLEM |
3 |atrial fibrillation |398 |416 |PROBLEM |
4 |late death |475 |484 |PROBLEM |
4 |cardiovascular-related re-hospitalization |490 |530 |PROBLEM |
6 |preoperative atrial fibrillation |655 |686 |PROBLEM |
7 |stroke |793 |798 |PROBLEM |
7 |renal failure |805 |817 |PROBLEM |
7 |atrial fibrillationand mortality |918 |949 |PROBLEM |
8 |preoperative atrial fibrillationwas |1055 |1089|PROBLEM |
+-----------+---------------------------------------------+-----+----+---------+
only showing top 20 rows

**As you can see, we got only `PROBLEM` entities from the text.**

### NER Visualization

We have sparknlp_display library for visualization. This library works with LightPipeline results.

In [0]:
sample_text = [pubMedDF.limit(3).collect()[i][0] for i in range(3)]

In [0]:
sample_text[1]

Out[18]: 'BACKGROUND: At present, it is one of the most important issues for the treatment of breast cancer to develop the standard therapy for patients previously treated with anthracyclines and taxanes. With the objective of determining the usefulnessof vinorelbine monotherapy in patients with advanced or recurrent breast cancerafter standard therapy, we evaluated the efficacy and safety of vinorelbine inpatients previously treated with anthracyclines and taxanes. METHODS: Vinorelbinewas administered at a dose level of 25 mg/m(2) intravenously on days 1 and 8 of a3 week cycle. Patients were given three or more cycles in the absence of tumorprogression. A maximum of nine cycles were administered. RESULTS: The responserate in 50 evaluable patients was 20.0% (10 out of 50; 95% confidence interval,10.0-33.7%). Responders plus those who had minor response (MR) or no change (NC) accounted for 58.0% [10 partial responses (PRs) + one MR + 18 NCs out of 50]. TheKaplan-Meier estimate (50% point) of time to progression (TTP) was 115.0 days.The response rate in the visceral organs was 17.3% (nine PRs out of 52). Themajor toxicity was myelosuppression, which was reversible and did not requirediscontinuation of treatment. CONCLUSION: The results of this study show thatvinorelbine monotherapy is useful in patients with advanced or recurrent breastcancer previously exposed to both anthracyclines and taxanes.'

In [0]:
ner_lp = LightPipeline(ner_model)
light_result = ner_lp.fullAnnotate(sample_text[1])

In [0]:
from sparknlp_display import NerVisualizer

visualiser = NerVisualizer()

vis = visualiser.display(light_result[0], label_col='ner_chunk', document_col='document', return_html=True)

# Change color of an entity label
#visualiser.set_label_colors({'PROBLEM':'#008080', 'TEST':'#800080', 'TREATMENT':'#808080'})
#visualiser.display(light_result[0], label_col='ner_chunk')

# Set label filter
# vis = visualiser.display(light_result, label_col='ner_chunk', document_col='document',
                   #labels=['PROBLEM','TEST','TREATMENT])
  
displayHTML(vis)

BACKGROUND: At present, it is one of the most important issues for the treatment TREATMENT of breast cancer PROBLEM to develop the standard therapy TREATMENT for patients previously treated with anthracyclines TREATMENT and taxanes TREATMENT . With the objective of determining the usefulnessof vinorelbine monotherapy TREATMENT in patients with advanced or recurrent breast cancerafter standard therapy TREATMENT , we evaluated the efficacy and safety of vinorelbine TREATMENT inpatients previously treated with anthracyclines TREATMENT and taxanes TREATMENT . METHODS: Vinorelbinewas TREATMENT administered at a dose level of 25 mg/m(2) intravenously on days 1 and 8 of a3 week cycle. Patients were given three or more cycles in the absence of tumorprogression TREATMENT . A maximum of nine cycles were administered. RESULTS: The responserate in 50 evaluable patients was 20.0% (10 out of 50; 95% confidence interval,10.0-33.7%). Responders plus those who had minor response ( MR) TEST or no change PROBLEM ( NC) TEST accounted for 58.0% [10 partial responses TEST ( PRs TEST ) + one MR TEST + 18 NCs out TEST of 50]. TheKaplan-Meier estimate TEST (50% point) of time to progression (TTP) was 115.0 days. The response rate in the visceral organs TEST was 17.3% (nine PRs out of 52). Themajor toxicity PROBLEM was myelosuppression PROBLEM , which was reversible and did not requirediscontinuation TREATMENT of treatment TREATMENT . CONCLUSION: The results of this study TEST show thatvinorelbine monotherapy TREATMENT is useful in patients with advanced or recurrent breastcancer TREATMENT previously exposed to both anthracyclines TREATMENT and taxanes TREATMENT .

**There are many NER models for different purposes in Spark NLP. Lets show what if we use `jsl_ner_wip_clinical` model that has about 80 different NER label.**

In [0]:
jsl_ner = MedicalNerModel.pretrained("jsl_ner_wip_clinical","en","clinical/models")\
        .setInputCols(["sentence","token","embeddings"])\
        .setOutputCol("ner")

jslPipeline = Pipeline(stages=[
        documentAssembler,
        sentenceDetector,
        tokenizer,
        word_embeddings,
        pos_tagger,
        jsl_ner,
        ner_converter])


empty_data = spark.createDataFrame([[""]]).toDF("text")

jsl_model = jslPipeline.fit(empty_data)

jsl_ner_wip_clinical download started this may take some time.
Approximate size to download 14.5 MB
[ | ][ / ][OK!]

In [0]:
jsl_lp = LightPipeline(jsl_model)
jsl_light_result = jsl_lp.fullAnnotate(sample_text[1])

In [0]:
from sparknlp_display import NerVisualizer

visualiser = NerVisualizer()

vis = visualiser.display(jsl_light_result[0], label_col='ner_chunk', document_col='document', return_html=True)

displayHTML(vis)

BACKGROUND: Section_Header At present, it is one of the most important issues for the treatment of breast cancer Oncological to develop the standard therapy for patients previously treated with anthracyclines Drug_Ingredient and taxanes Drug_Ingredient . With the objective of determining the usefulnessof vinorelbine monotherapy Drug_Ingredient in patients with advanced or recurrent Modifier breast cancerafter standard therapy Procedure , we evaluated the efficacy and safety of vinorelbine Drug_Ingredient inpatients previously treated with anthracyclines Drug_Ingredient and taxanes Drug_Ingredient . METHODS: Section_Header Vinorelbinewas administered at a dose level of 25 Strength mg/m(2) intravenously Route on days 1 and 8 of a3 week cycle Frequency . Patients were given three or more cycles Frequency in the absence of tumorprogression. A maximum of nine cycles Dosage were administered. RESULTS: Section_Header The responserate in 50 evaluable patients was 20.0% (10 out of 50; 95% confidence interval,10.0-33.7%). Responders plus those who had minor response Test_Result ( MR Test ) or no change Test_Result ( NC Test ) accounted for 58.0% [10 partial responses Test ( PRs Test ) + one MR Test + 18 Test_Result NCs Test out of 50] Test_Result . TheKaplan-Meier estimate (50% point) of time to progression ( TTP Disease_Syndrome_Disorder ) was 115.0 days Duration .The response rate in the visceral organs Internal_organ_or_component was 17.3% Test_Result (nine PRs out of 52). Themajor toxicity Disease_Syndrome_Disorder was myelosuppression Symptom , which was reversible Modifier and did not requirediscontinuation of treatment. CONCLUSION: Section_Header The results of this study show thatvinorelbine monotherapy Drug_Ingredient is useful in patients with advanced or recurrent Modifier breastcancer Symptom previously exposed to both anthracyclines Drug_Ingredient and taxanes Drug_Ingredient .

**If you want to go over more about NER, you can check this comprehensive notebook :**

https://github.com/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/Certification_Trainings/Healthcare/databricks_notebooks/1.Clinical_Named_Entity_Recognition_Model_v3.0.ipynb

## 2. Clinical Assertion

Now we will check the assertion status of the clinical entities. We will use `ner_clinical_large` model for NER detection, and `assertion_dl` model for checking the assertion status of detected entities. While doing that, we will use the same pipeline that we created fot detecting NER.

In [0]:
# Assertion model trained on i2b2 (sampled from MIMIC) dataset
# coming from sparknlp_jsl.annotator !!
clinical_assertion = AssertionDLModel.pretrained("assertion_dl", "en", "clinical/models") \
    .setInputCols(["sentence", "ner_chunk", "embeddings"]) \
    .setOutputCol("assertion")
    
assertionPipeline = Pipeline(stages=[
    nerPipeline,
    clinical_assertion
    ])

empty_data = spark.createDataFrame([[""]]).toDF("text")

assertion_model = assertionPipeline.fit(empty_data)

assertion_dl download started this may take some time.
Approximate size to download 1.3 MB
[ | ][OK!]

**This time we will use LightPipeline while implementing.**

In [0]:
sample_text[0]

Out[25]: 'The human KCNJ9 (Kir 3.3, GIRK3) is a member of the G-protein-activated inwardly rectifying potassium (GIRK) channel family. Here we describe the genomicorganization of the KCNJ9 locus on chromosome 1q21-23 as a candidate gene forType II diabetes mellitus in the Pima Indian population. The gene spansapproximately 7.6 kb and contains one noncoding and two coding exons separated byapproximately 2.2 and approximately 2.6 kb introns, respectively. We identified14 single nucleotide polymorphisms (SNPs), including one that predicts aVal366Ala substitution, and an 8 base-pair (bp) insertion/deletion. Ourexpression studies revealed the presence of the transcript in various humantissues including pancreas, and two major insulin-responsive tissues: fat andskeletal muscle. The characterization of the KCNJ9 gene should facilitate furtherstudies on the function of the KCNJ9 protein and allow evaluation of thepotential role of the locus in Type II diabetes.'

In [0]:
assertion_light = LightPipeline(assertion_model)

**We can use `annotate` method to get faster results for short sentences.**

In [0]:
assertion_anno_res = assertion_light.annotate(sample_text[0])

In [0]:
assertion_anno_res.keys()

Out[28]: dict_keys(['document', 'ner_chunk', 'assertion', 'token', 'ner', 'embeddings', 'pos_tags', 'sentence'])

**Lets create a pandas dataframe to see our results obviously.**

In [0]:
pd.DataFrame(list(zip(assertion_anno_res["ner_chunk"], assertion_anno_res["assertion"])), columns=["ner_chunk", "assertion"])

Out[29]:

,ner_chunk,assertion
0,the G-protein-activated inwardly rectifying potassium (GIRK,conditional
1,the genomicorganization,present
2,a candidate gene forType II diabetes mellitus,present
3,byapproximately,present
4,single nucleotide polymorphisms,present
5,aVal366Ala substitution,present
6,an 8 base-pair,present
7,insertion/deletion,absent
8,Ourexpression studies,present
9,the transcript in various humantissues,present


**This time we will use `fullAnnotate` method on our text to get metadata results.**

In [0]:
assertion_result = assertion_light.fullAnnotate(sample_text[0])[0]

In [0]:
assertion_result.keys()

Out[31]: dict_keys(['document', 'ner_chunk', 'assertion', 'token', 'ner', 'embeddings', 'pos_tags', 'sentence'])

In [0]:
chunks=[]
entities=[]
status=[]

for n,m in zip(assertion_result['ner_chunk'],assertion_result['assertion']):

    chunks.append(n.result)
    entities.append(n.metadata['entity']) 
    status.append(m.result)

df = pd.DataFrame({'chunks':chunks, 'entities':entities, 'assertion':status})

In [0]:
df

Out[33]:

,chunks,entities,assertion
0,the G-protein-activated inwardly rectifying potassium (GIRK,TREATMENT,conditional
1,the genomicorganization,TREATMENT,present
2,a candidate gene forType II diabetes mellitus,PROBLEM,present
3,byapproximately,TREATMENT,present
4,single nucleotide polymorphisms,TREATMENT,present
5,aVal366Ala substitution,PROBLEM,present
6,an 8 base-pair,PROBLEM,present
7,insertion/deletion,PROBLEM,absent
8,Ourexpression studies,TEST,present
9,the transcript in various humantissues,PROBLEM,present


**Also we can filter assertion results by using `AssertionFilterer` annotator. We will use the same pipeline that we vreated before to get the assertions. We will try to get only `present` assertions.**

In [0]:
assertion_filterer = AssertionFilterer()\
      .setInputCols("sentence","ner_chunk","assertion")\
      .setOutputCol("assertion_filtered")\
      .setWhiteList(["present"])

assertionFilteredPipeline = Pipeline(stages=[
    assertionPipeline,
    assertion_filterer
    ])

empty_data = spark.createDataFrame([[""]]).toDF("text")

assertion_filtered_model = assertionFilteredPipeline.fit(empty_data)

In [0]:
assertion_filtered_light = LightPipeline(assertion_filtered_model)

In [0]:
assertion_filtered_result = assertion_filtered_light.fullAnnotate(sample_text[0])[0]

In [0]:
assertion_filtered_result.keys()

Out[37]: dict_keys(['assertion_filtered', 'document', 'ner_chunk', 'assertion', 'token', 'ner', 'embeddings', 'pos_tags', 'sentence'])

In [0]:
assertion_filtered_result["assertion_filtered"]

Out[38]: [Annotation(chunk, 142, 164, the genomicorganization, {'entity': 'TREATMENT', 'sentence': '1', 'chunk': '1', 'confidence': '0.96985'}),
 Annotation(chunk, 210, 254, a candidate gene forType II diabetes mellitus, {'entity': 'PROBLEM', 'sentence': '1', 'chunk': '2', 'confidence': '0.7840571'}),
 Annotation(chunk, 380, 394, byapproximately, {'entity': 'TREATMENT', 'sentence': '2', 'chunk': '3', 'confidence': '0.9692'}),
 Annotation(chunk, 464, 494, single nucleotide polymorphisms, {'entity': 'TREATMENT', 'sentence': '3', 'chunk': '4', 'confidence': '0.79413337'}),
 Annotation(chunk, 532, 554, aVal366Ala substitution, {'entity': 'PROBLEM', 'sentence': '3', 'chunk': '5', 'confidence': '0.85905004'}),
 Annotation(chunk, 561, 574, an 8 base-pair, {'entity': 'PROBLEM', 'sentence': '3', 'chunk': '6', 'confidence': '0.73810005'}),
 Annotation(chunk, 601, 621, Ourexpression studies, {'entity': 'TEST', 'sentence': '4', 'chunk': '8', 'confidence': '0.8843'}),
 Annotation(chunk, 648, 685, the transcript in various humantissues, {'entity': 'PROBLEM', 'sentence': '4', 'chunk': '9', 'confidence': '0.92882'}),
 Annotation(chunk, 830, 843, furtherstudies, {'entity': 'PROBLEM', 'sentence': '5', 'chunk': '11', 'confidence': '0.9981'}),
 Annotation(chunk, 864, 880, the KCNJ9 protein, {'entity': 'TREATMENT', 'sentence': '5', 'chunk': '12', 'confidence': '0.7142'}),
 Annotation(chunk, 940, 955, Type II diabetes, {'entity': 'PROBLEM', 'sentence': '5', 'chunk': '14', 'confidence': '0.9191'})]

Here is the `present` entities.

In [0]:
chunks=[]
entities=[]


for n in assertion_filtered_result['assertion_filtered']:

    chunks.append(n.result)
    entities.append(n.metadata['entity']) 


filtered_df = pd.DataFrame({'chunks':chunks, 'entities':entities})

filtered_df

Out[39]:

,chunks,entities
0,the genomicorganization,TREATMENT
1,a candidate gene forType II diabetes mellitus,PROBLEM
2,byapproximately,TREATMENT
3,single nucleotide polymorphisms,TREATMENT
4,aVal366Ala substitution,PROBLEM
5,an 8 base-pair,PROBLEM
6,Ourexpression studies,TEST
7,the transcript in various humantissues,PROBLEM
8,furtherstudies,PROBLEM
9,the KCNJ9 protein,TREATMENT


### Assertion Visualization

We can visualize the assertion status of detected entities by using `AssertionVisualizer` module of `sparknlp_display` library.

In [0]:
from sparknlp_display import AssertionVisualizer

assertion_vis = AssertionVisualizer()

## To set custom label colors:
assertion_vis.set_label_colors({'TREATMENT':'#008080', 'PROBLEM':'#800080'}) #set label colors by specifying hex codes

vis = assertion_vis.display(assertion_result, 
                            label_col = 'ner_chunk', 
                            assertion_col = 'assertion',
                            document_col = 'document' ,
                            return_html=True
                      )

displayHTML(vis)

The human KCNJ9 (Kir 3.3, GIRK3) is a member of the G-protein-activated inwardly rectifying potassium (GIRK TREATMENT conditional ) channel family. Here we describe the genomicorganization TREATMENT present of the KCNJ9 locus on chromosome 1q21-23 as a candidate gene forType II diabetes mellitus PROBLEM present in the Pima Indian population. The gene spansapproximately 7.6 kb and contains one noncoding and two coding exons separated byapproximately TREATMENT present 2.2 and approximately 2.6 kb introns, respectively. We identified14 single nucleotide polymorphisms TREATMENT present (SNPs), including one that predicts aVal366Ala substitution PROBLEM present , and an 8 base-pair PROBLEM present (bp) insertion/deletion PROBLEM absent . Ourexpression studies TEST present revealed the presence of the transcript in various humantissues PROBLEM present including pancreas, and two major insulin-responsive tissues: fat andskeletal muscle PROBLEM possible . The characterization of the KCNJ9 gene should facilitate furtherstudies PROBLEM present on the function of the KCNJ9 protein TREATMENT present and allow evaluation TEST possible of thepotential role of the locus in Type II diabetes PROBLEM present .

**If you want to go more over about assertion model examples, you can check this comprehensive notebook :**

https://github.com/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/Certification_Trainings/Healthcare/2.Clinical_Assertion_Model.ipynb

## 3. Relation Extraction

In this section, we will show an example of relation extraction models. We will use the same NER pipeline that we created before to extract clinical entities and `re_clinical` model to extract relations between these entities. The set of relations defined in the 2010 i2b2 relation challenge:

**TrIP:** A certain treatment has improved or cured a medical problem (eg, ‘infection resolved with antibiotic course’)

**TrWP:** A patient's medical problem has deteriorated or worsened because of or in spite of a treatment being administered (eg, ‘the tumor was growing despite the drain’)

**TrCP:** A treatment caused a medical problem (eg, ‘penicillin causes a rash’)

**TrAP:** A treatment administered for a medical problem (eg, ‘Dexamphetamine for narcolepsy’)

**TrNAP:** The administration of a treatment was avoided because of a medical problem (eg, ‘Ralafen which is contra-indicated because of ulcers’)

**TeRP:** A test has revealed some medical problem (eg, ‘an echocardiogram revealed a pericardial effusion’)

**TeCP:** A test was performed to investigate a medical problem (eg, ‘chest x-ray done to rule out pneumonia’)

**PIP:** Two problems are related to each other (eg, ‘Azotemia presumed secondary to sepsis’)

In [0]:
dependency_parser = DependencyParserModel()\
    .pretrained("dependency_conllu", "en")\
    .setInputCols(["sentence", "pos_tags", "token"])\
    .setOutputCol("dependencies")

clinical_re_Model = RelationExtractionModel()\
    .pretrained("re_clinical", "en", 'clinical/models')\
    .setInputCols(["embeddings", "pos_tags", "ner_chunk", "dependencies"])\
    .setOutputCol("relations")\
    .setMaxSyntacticDistance(4)\
    #.setRelationPairs(["problem-test", "problem-treatment"]) we can set the possible relation pairs (if not set, all the relations will be calculated)

relPipeline = Pipeline(stages=[
    nerPipeline,
    dependency_parser,
    clinical_re_Model
])


empty_data = spark.createDataFrame([[""]]).toDF("text")

rel_model = relPipeline.fit(empty_data)

dependency_conllu download started this may take some time.
Approximate size to download 16.7 MB
[ | ][ / ][ — ][ \ ][ | ][OK!]
re_clinical download started this may take some time.
Approximate size to download 6 MB
[ | ][OK!]

In [0]:
rel_model.stages

Out[42]: [PipelineModel_511d5b6ec81d,
 dependency_d5a8da6c9093,
 RelationExtractionModel_9c255241fec3]

In [0]:
sample_text[1]

Out[43]: 'BACKGROUND: At present, it is one of the most important issues for the treatment of breast cancer to develop the standard therapy for patients previously treated with anthracyclines and taxanes. With the objective of determining the usefulnessof vinorelbine monotherapy in patients with advanced or recurrent breast cancerafter standard therapy, we evaluated the efficacy and safety of vinorelbine inpatients previously treated with anthracyclines and taxanes. METHODS: Vinorelbinewas administered at a dose level of 25 mg/m(2) intravenously on days 1 and 8 of a3 week cycle. Patients were given three or more cycles in the absence of tumorprogression. A maximum of nine cycles were administered. RESULTS: The responserate in 50 evaluable patients was 20.0% (10 out of 50; 95% confidence interval,10.0-33.7%). Responders plus those who had minor response (MR) or no change (NC) accounted for 58.0% [10 partial responses (PRs) + one MR + 18 NCs out of 50]. TheKaplan-Meier estimate (50% point) of time to progression (TTP) was 115.0 days.The response rate in the visceral organs was 17.3% (nine PRs out of 52). Themajor toxicity was myelosuppression, which was reversible and did not requirediscontinuation of treatment. CONCLUSION: The results of this study show thatvinorelbine monotherapy is useful in patients with advanced or recurrent breastcancer previously exposed to both anthracyclines and taxanes.'

In [0]:
rel_light = LightPipeline(rel_model)
relation_res = rel_light.fullAnnotate(sample_text[1])[0]

In [0]:
relation_res.keys()

Out[45]: dict_keys(['document', 'ner_chunk', 'token', 'relations', 'ner', 'embeddings', 'pos_tags', 'dependencies', 'sentence'])

In [0]:
rel_pairs=[]
  
for rel in relation_res["relations"]:
    rel_pairs.append((
          rel.result, 
          rel.metadata['entity1'], 
          rel.metadata['entity1_begin'],
          rel.metadata['entity1_end'],
          rel.metadata['chunk1'], 
          rel.metadata['entity2'],
          rel.metadata['entity2_begin'],
          rel.metadata['entity2_end'],
          rel.metadata['chunk2'], 
          rel.metadata['confidence']
      ))

rel_df = pd.DataFrame(rel_pairs, columns=['relation','entity1','entity1_begin','entity1_end','chunk1','entity2','entity2_begin','entity2_end','chunk2', 'confidence'])
rel_df[rel_df.relation!="O"]

Out[46]:

,relation,entity1,entity1_begin,entity1_end,chunk1,entity2,entity2_begin,entity2_end,chunk2,confidence
1,TrWP,PROBLEM,84,96,breast cancer,TREATMENT,109,128,the standard therapy,0.6738779
3,TeRP,TREATMENT,109,128,the standard therapy,TREATMENT,186,192,taxanes,0.9952891
5,TeRP,TREATMENT,229,268,the usefulnessof vinorelbine monotherapy,TREATMENT,287,343,advanced or recurrent breast cancerafter standard therapy,0.9998728
14,TeRP,PROBLEM,1110,1126,Themajor toxicity,PROBLEM,1132,1147,myelosuppression,0.7527134
15,TeRP,PROBLEM,1110,1126,Themajor toxicity,TREATMENT,1183,1204,requirediscontinuation,0.7808279
16,TeRP,PROBLEM,1110,1126,Themajor toxicity,TREATMENT,1209,1217,treatment,0.9991315
17,TrWP,PROBLEM,1132,1147,myelosuppression,TREATMENT,1183,1204,requirediscontinuation,0.8995943
21,TeRP,TREATMENT,1263,1289,thatvinorelbine monotherapy,TREATMENT,1330,1351,recurrent breastcancer,0.9413687
22,TeRP,TREATMENT,1330,1351,recurrent breastcancer,TREATMENT,1380,1393,anthracyclines,0.9842151
23,TeRP,TREATMENT,1330,1351,recurrent breastcancer,TREATMENT,1399,1405,taxanes,0.9980427


### Relation Visualization

We can visualize relations between entities by using `RelationExtractionVisualizer` module of `sparknlp_display` lìbrary.

In [0]:
from sparknlp_display import RelationExtractionVisualizer

re_vis = RelationExtractionVisualizer()

vis = re_vis.display(relation_res,
                     relation_col = 'relations',
                     document_col = 'document',
                     show_relations=True,
                     return_html=True)

displayHTML(vis)

BACKGROUND: At present, it is one of the most important issues for the treatment of breast cancer PROBLEM to develop the standard therapy TREATMENT for patients previously treated with anthracyclines and taxanes TREATMENT . With the objective of determining the usefulnessof vinorelbine monotherapy TREATMENT in patients with advanced or recurrent breast cancerafter standard therapy TREATMENT , we evaluated the efficacy and safety of vinorelbine inpatients previously treated with anthracyclines and taxanes. METHODS: Vinorelbinewas administered at a dose level of 25 mg/m(2) intravenously on days 1 and 8 of a3 week cycle. Patients were given three or more cycles in the absence of tumorprogression. A maximum of nine cycles were administered. RESULTS: The responserate in 50 evaluable patients was 20.0% (10 out of 50; 95% confidence interval,10.0-33.7%). Responders plus those who had minor response (MR) or no change (NC) accounted for 58.0% [10 partial responses (PRs) + one MR + 18 NCs out of 50]. TheKaplan-Meier estimate (50% point) of time to progression (TTP) was 115.0 days.The response rate in the visceral organs was 17.3% (nine PRs out of 52). Themajor toxicity PROBLEM was myelosuppression PROBLEM , which was reversible and did not requirediscontinuation TREATMENT of treatment TREATMENT . CONCLUSION: The results of this study show thatvinorelbine monotherapy TREATMENT is useful in patients with advanced or recurrent breastcancer TREATMENT previously exposed to both anthracyclines TREATMENT and taxanes TREATMENT . <polyline fill="none" points="283.0,455.0 282.6020408163265,454.3367346938775 282.204081632653,453.67346938775506 281.8061224489796,453.01020408163265 281.4081632653061,452.3469387755102 281.0102040816326,451.68367346938777 280.6122448979591,451.0204081632653 280.2142857142858,450.35714285714295 279.81632653061234,449.69387755102053 279.4183673469388,449.030612244898 279.02040816326524,448.3673469387755 278.6224489795918,447.7040816326531 278.2244897959183,447.04081632653055 277.8265306122449,446.3775510204082 277.42857142857144,445.7142857142858 277.030612244898,445.05102040816337 276.6326530612245,444.38775510204084 276.2346938775511,443.72448979591843 275.83673469387753,443.0612244897959 275.43877551020404,442.3979591836735 275.0408163265306,441.7346938775509 274.6428571428571,441.07142857142856 274.2448979591837,440.40816326530614 273.8469387755102,439.7448979591836 273.44897959183675,439.08163265306126 273.05102040816325,438.4183673469388 272.65306122448976,437.7551020408163 272.25510204081627,437.09183673469397 271.8571428571429,436.4285714285715 271.4591836734694,435.765306122449 271.0612244897959,435.10204081632656 270.6632653061224,434.43877551020415 270.265306122449,433.7755102040816 269.86734693877554,433.1122448979593 269.46938775510205,432.44897959183675 269.07142857142856,431.78571428571433 268.67346938775506,431.1224489795919 268.2755102040816,430.4591836734694 267.8775510204082,429.795918367347 267.4795918367347,429.13265306122446 267.0816326530612,428.46938775510205 266.68367346938777,427.8061224489796 266.28571428571433,427.1428571428572 265.88775510204084,426.4795918367347 265.48979591836735,425.8163265306123 265.09183673469386,425.1530612244898 264.6938775510204,424.48979591836735 264.2959183673469,423.8265306122449 263.89795918367344,423.1632653061224 263.5,422.5 263.5,422.5 263.1020408163265,421.83673469387753 262.7040816326531,421.173469387755 262.3061224489796,420.51020408163265 261.90816326530614,419.8469387755102 261.51020408163265,419.1836734693877 261.11224489795916,418.52040816326524 260.71428571428584,417.857142857143 260.3163265306123,417.1938775510205 259.9183673469388,416.530612244898 259.52040816326524,415.8673469387755 259.1224489795918,415.20408163265296 258.7244897959184,414.54081632653055 258.3265306122449,413.87755102040813 257.92857142857144,413.21428571428567 257.530612244898,412.55102040816325 257.1326530612245,411.88775510204084 256.734693877551,411.2244897959184 256.3367346938

**If you want to go more over about relation extraction model examples, you can check this comprehensive notebook :**

https://github.com/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/Certification_Trainings/Healthcare/10.Clinical_Relation_Extraction.ipynb

## 4. Entity Resolution

There are many entity resolution models for different kinds of purposes in Spark NLP. But mainly, we can collect these models in two categories:

* Chunk Entity Resolver Models
* Sentence Entity Resolver Models

Here are some of the resolver models in Spark NLP:

- sbiobertresolve_icd10cm 
- sbiobertresolve_icd10cm_augmented
- sbiobertresolve_icd10cm_slim_normalized
- sbiobertresolve_icd10cm_slim_billable_hcc
- sbertresolve_icd10cm_slim_billable_hcc_med
- sbiobertresolve_icd10pcs
- sbiobertresolve_snomed_findings (with clinical_findings concepts from CT version)
- sbiobertresolve_snomed_findings_int  (with clinical_findings concepts from INT version)
- sbiobertresolve_snomed_auxConcepts (with Morph Abnormality, Procedure, Substance, Physical Object, Body Structure concepts from CT version)
- sbiobertresolve_snomed_auxConcepts_int  (with Morph Abnormality, Procedure, Substance, Physical Object, Body Structure concepts from INT version)
- sbiobertresolve_rxnorm
- sbiobertresolve_rxcui
- sbiobertresolve_icdo
- sbiobertresolve_cpt
- sbiobertresolve_loinc
- sbiobertresolve_HPO
- sbiobertresolve_umls_major_concepts
- sbiobertresolve_umls_findings
- ...

We will use the same NER pipeline and `sbiobertresolve_icd10cm_slim_billable_hcc` ICD10 CM entity resolver model.

In [0]:
c2doc = Chunk2Doc()\
      .setInputCols("ner_chunk")\
      .setOutputCol("ner_chunk_doc") 

sbert_embedder = BertSentenceEmbeddings\
      .pretrained("sbert_jsl_medium_uncased",'en','clinical/models')\
      .setInputCols(["ner_chunk_doc"])\
      .setOutputCol("sbert_embeddings")

icd_resolver = SentenceEntityResolverModel.pretrained("sbertresolve_icd10cm_slim_billable_hcc_med","en", "clinical/models") \
      .setInputCols(["ner_chunk", "sbert_embeddings"]) \
      .setOutputCol("icd10_code")\
      .setDistanceFunction("EUCLIDEAN")


resolverPipeline = Pipeline(stages=[
        nerPipeline,
        c2doc,
        sbert_embedder,
        icd_resolver
    
])

empty_data = spark.createDataFrame([[""]]).toDF("text")
resolver_model = resolverPipeline.fit(empty_data)

sbert_jsl_medium_uncased download started this may take some time.
Approximate size to download 146.8 MB
[ | ][ / ][ — ][ \ ][ | ][OK!]
sbertresolve_icd10cm_slim_billable_hcc_med download started this may take some time.
Approximate size to download 574.3 MB
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][OK!]

In [0]:
res_light = LightPipeline(resolver_model)

In [0]:
res_anno = res_light.annotate("bladder cancer")

In [0]:
res_anno

Out[52]: {'document': ['bladder cancer'],
 'ner_chunk': ['bladder cancer'],
 'token': ['bladder', 'cancer'],
 'sbert_embeddings': ['bladder cancer'],
 'ner': ['B-PROBLEM', 'I-PROBLEM'],
 'embeddings': ['bladder', 'cancer'],
 'pos_tags': ['JJR', 'NN'],
 'icd10_code': ['C671'],
 'ner_chunk_doc': ['bladder cancer'],
 'sentence': ['bladder cancer']}

In [0]:
list(zip(res_anno["ner_chunk"], res_anno["icd10_code"]))

Out[53]: [('bladder cancer', 'C671')]

In [0]:
resolver_res = res_light.fullAnnotate(sample_text[1])[0]

In [0]:
resolver_res.keys()

Out[55]: dict_keys(['document', 'ner_chunk', 'token', 'sbert_embeddings', 'ner', 'embeddings', 'pos_tags', 'icd10_code', 'ner_chunk_doc', 'sentence'])

In [0]:
chunks = []
codes = []
begin = []
end = []
resolutions= []
all_distances = []
all_codes= []
all_cosines = []
all_k_aux_labels= []
confidence = []
entity = []

for chunk, code in zip(resolver_res['ner_chunk'], resolver_res["icd10_code"]):

    begin.append(chunk.begin)
    entity.append(chunk.metadata['entity'])
    end.append(chunk.end)
    chunks.append(chunk.result)
    codes.append(code.result) 
    confidence.append(code.metadata['confidence'])
    all_codes.append(code.metadata['all_k_results'].split(':::'))
    resolutions.append(code.metadata['all_k_resolutions'].split(':::'))
    all_distances.append(code.metadata['all_k_distances'].split(':::'))
    all_cosines.append(code.metadata['all_k_cosine_distances'].split(':::'))
    all_k_aux_labels.append(code.metadata['all_k_aux_labels'].split(':::'))
    
df = pd.DataFrame({'chunks':chunks, 'entity':entity, 'begin': begin, 'end':end, 'code':codes, 'all_codes':all_codes, 
                   'resolutions':resolutions, 'all_k_aux_labels':all_k_aux_labels,'all_distances':all_cosines})



df['billable'] = df['all_k_aux_labels'].apply(lambda x: [i.split('||')[0] for i in x])
df['hcc_status'] = df['all_k_aux_labels'].apply(lambda x: [i.split('||')[1] for i in x])
df['hcc_score'] = df['all_k_aux_labels'].apply(lambda x: [i.split('||')[2] for i in x])
df['confidence'] = confidence

df = df.drop(['all_k_aux_labels'], axis=1)

In [0]:
pd.set_option("display.max_colwidth", 100)

In [0]:
df

Out[58]:

,chunks,entity,begin,end,code,all_codes,resolutions,all_distances,billable,hcc_status,hcc_score,confidence
0,the treatment,TREATMENT,67,79,Z7689,"[Z7689, Z789, F4329, N5313, Z3141, Z37, Q438, E45, Z9189, T66XXXA, Z7189]","[response to treatment [Persons encountering health services in other specified circumstances], ...","[0.2620, 0.2690, 0.2773, 0.2850, 0.2940, 0.2921, 0.3030, 0.2976, 0.3103, 0.3050, 0.2962]","[1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1]","[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 21, 0, 0, 0]",0.1487
1,breast cancer,PROBLEM,84,96,C50919,"[C50919, Z1239, C50911, C4452, D0590, D493, C61, C44501, Z853, C50819, C50111, C50929, C50921, C...","[breast cancer [Malignant neoplasm of unspecified site of unspecified female breast], screening ...","[0.0000, 0.1055, 0.1108, 0.1151, 0.1247, 0.1303, 0.1323, 0.1391, 0.1372, 0.1420, 0.1439, 0.1440,...","[1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0]","[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1]","[0, 0, 0, 0, 0, 0, 12, 0, 0, 0, 0, 0, 0, 12, 0, 0, 0, 9, 12]",0.9984
2,the standard therapy,TREATMENT,109,128,Z789,"[Z789, Z7689, Z9289, Z5181, Z7189]","[complete therapeutic response [Other specified health status], ideal weight discussed (regime/t...","[0.2608, 0.2806, 0.2919, 0.2893, 0.3101]","[1, 1, 1, 1, 1]","[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]",0.2919
3,anthracyclines,TREATMENT,167,180,A220,"[A220, A229, A22, A222, L940, Z1629, B999, D239, A599, A227, Q821, A5272, H5054, L998, N897, A22...","[skin anthrax [Cutaneous anthrax], anthrax infection [Anthrax, unspecified], anthrax [Anthrax], ...","[0.2478, 0.2633, 0.2688, 0.2688, 0.2868, 0.2857, 0.2857, 0.2944, 0.2948, 0.2823, 0.2905, 0.2970,...","[1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 115, 6, 0, 0, 0, 0, 0]",0.0817
4,taxanes,TREATMENT,186,192,H1511,"[H1511, H15119, D4989, C7220, Q103, C781, L293, L291, I722, P2810, I714, R198, Q702, Q7649, L812...","[episcleritis periodica fugax [Episcleritis periodica fugax], episcleritis periodica fugax [Epis...","[0.3039, 0.3039, 0.2873, 0.2735, 0.2948, 0.3006, 0.2971, 0.2971, 0.2937, 0.3039, 0.3005, 0.2809,...","[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1]","[0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1]","[0, 0, 0, 10, 0, 8, 0, 0, 108, 0, 108, 0, 0, 0, 0, 108, nan, 111, 11, 11, 0, 0, 11, 108, 107]",0.0579
5,the usefulnessof vinorelbine monotherapy,TREATMENT,229,268,Z7189,"[Z7189, Z7689, Z4931, L049, T464X4S, Z713, K5900, Z3183, Z4932, F1621, F1921, T43214, F18921, Z4...","[accutane treatment counseling [Other specified counseling], incretin mimetic therapy started (s...","[0.1702, 0.1904, 0.2058, 0.2281, 0.2153, 0.2091, 0.2091, 0.2194, 0.2257, 0.2462, 0.2462, 0.2356,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1]","[0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0]","[0, 0, 134, 0, 0, 0, 0, 0, 134, 55, 55, 0, 0, 134, 0, 0]",0.1363
6,advanced or recurrent breast cancerafter standard therapy,TREATMENT,287,343,Z1239,"[Z1239, Z8541, Z923, Z125, C802, T8649, R9721, T86818, Z1503, O3412, O3413, C5091, O3410, O3411,...",[screening exam for breast cancer [Encounter for other screening for malignant neoplasm of breas...,"[0.1534, 0.1573, 0.1573, 0.1514, 0.1550, 0.1550, 0.1631, 0.1582, 0.1690, 0.1640, 0.1650, 0.1732,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1]","[0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0]","[0, 0, 0, 0, 12, 186, 0, 0, 0, 0, 0, 12, 0, 0, 186, 8, 0, 11, 0, 0, 0, 0]",0.0606
7,vinorelbine,TREATMENT,386,396,E7201,"[E7201, H5353, N8181, O418X, L708, F488, R7989, S0082, S0082XA, I7389, S0082XD, J3489, I898, Y95...","[cystinuria, type 1 [Cystinuria], deuteranomaly [Deuteranomaly], perineocele [Perineocele], find...","[0.2897, 0.2942, 0.2888, 0.3018, 0.2972, 0.3220, 0.3120, 0.3416, 0.3416, 0.3567, 0.3461, 0.320

**Lets check the confidence level > 0.5 results**

In [0]:
df[df.confidence.astype(float) > 0.5]

Out[59]:

,chunks,entity,begin,end,code,all_codes,resolutions,all_distances,billable,hcc_status,hcc_score,confidence
1,breast cancer,PROBLEM,84,96,C50919,"[C50919, Z1239, C50911, C4452, D0590, D493, C61, C44501, Z853, C50819, C50111, C50929, C50921, C...","[breast cancer [Malignant neoplasm of unspecified site of unspecified female breast], screening ...","[0.0000, 0.1055, 0.1108, 0.1151, 0.1247, 0.1303, 0.1323, 0.1391, 0.1372, 0.1420, 0.1439, 0.1440,...","[1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0]","[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1]","[0, 0, 0, 0, 0, 0, 12, 0, 0, 0, 0, 0, 0, 12, 0, 0, 0, 9, 12]",0.9984
19,TheKaplan-Meier estimate,TEST,956,979,Z7689,"[Z7689, D720, R700]",[seen by endocrinology service (finding) [Persons encountering health services in other specifie...,"[0.2599, 0.3172, 0.3300]","[1, 1, 1]","[0, 1, 0]","[0, 47, 0]",0.5512


### Entity Resolution Visualization

In [0]:
from sparknlp_display import EntityResolverVisualizer

er_vis = EntityResolverVisualizer()


## To set custom label colors:
er_vis.set_label_colors({'TREATMENT':'#800080', 'PROBLEM':'#77b5fe'}) #set label colors by specifying hex codes

vis = er_vis.display(resolver_res, 
                     label_col='ner_chunk', 
                     resolution_col = 'icd10_code',
                     document_col='document',
                     return_html=True)

displayHTML(vis)

BACKGROUND: At present, it is one of the most important issues for the treatment TREATMENT Z7689 response to treatment [Persons encountering health services in other specified circumstances] of breast cancer PROBLEM C50919 breast cancer [Malignant neoplasm of unspecified site of unspecified female breast] to develop the standard therapy TREATMENT Z789 complete therapeutic response [Other specified health status] for patients previously treated with anthracyclines TREATMENT A220 skin anthrax [Cutaneous anthrax] and taxanes TREATMENT H1511 episcleritis periodica fugax [Episcleritis periodica fugax] . With the objective of determining the usefulnessof vinorelbine monotherapy TREATMENT Z7189 accutane treatment counseling [Other specified counseling] in patients with advanced or recurrent breast cancerafter standard therapy TREATMENT Z1239 screening exam for breast cancer [Encounter for other screening for malignant neoplasm of breast] , we evaluated the efficacy and safety of vinorelbine TREATMENT E7201 cystinuria, type 1 [Cystinuria] inpatients previously treated with anthracyclines TREATMENT A220 skin anthrax [Cutaneous anthrax] and taxanes TREATMENT H1511 episcleritis periodica fugax [Episcleritis periodica fugax] . METHODS: Vinorelbinewas TREATMENT Z7689 under care of cmhn [Persons encountering health services in other specified circumstances] administered at a dose level of 25 mg/m(2) intravenously on days 1 and 8 of a3 week cycle. Patients were given three or more cycles in the absence of tumorprogression TREATMENT R3989 pseudoemission [Other symptoms and signs involving the genitourinary system] . A maximum of nine cycles were administered. RESULTS: The responserate in 50 evaluable patients was 20.0% (10 out of 50; 95% confidence interval,10.0-33.7%). Responders plus those who had minor response ( MR) TEST N469 primary male infertility [Male infertility, unspecified] or no change PROBLEM F6089 personality change [Other specific personality disorders] ( NC) TEST Z30431 iud, surveillance [Encounter for routine checking of intrauterine contraceptive device] accounted for 58.0% [10 partial responses TEST Z789 partial response to treatment [Other specified health status] ( PRs TEST Z30431 iud, surveillance [Encounter for routine checking of intrauterine contraceptive device] ) + one MR TEST H360 dr [Retinal disorders in diseases classified elsewhere] + 18 NCs out TEST Z789 patient in remission [Other specified health status] of 50]. TheKaplan-Meier estimate TEST Z7689 seen by endocrinology service (finding) [Persons encountering health services in other specified circumstances] (50% point) of time to progression (TTP) was 115.0 days. The response rate in the visceral organs TEST S278 injury of other specified intrathoracic organs [Injury of other specified intrathoracic organs] was 17.3% (nine PRs out of 52). Themajor toxicity PROBLEM T460X1A digoxin toxicity [Poisoning by cardiac-stimulant glycosides and drugs of similar action, accidental (unintentional), initial encounter] was myelosuppression PROBLEM D759 myelosuppression [Disease of blood and blood-forming organs, unspecified] , which was reversible and did not requirediscontinuation TREATMENT N3949 other specified urinary incontinence [Other specified urinary incontinence] of treatment TREATMENT Z9289 treatment given (regime/therapy) [Personal history of other medical treatment] . CONCLUSION: The results of this study TEST Z7689 patient in local study [Persons encountering health services in other specified circumstances] show thatvinorelbine monotherapy TREATMENT Z7689 incretin mimetic therapy started (situation) [Persons encountering health services in other specified circumstances] is useful in patients with advanced or recurrent breastcancer TREATMENT I830 recurrent venous leg ulcer [Varicose veins of lower extremities with ulcer] previously exposed to both anthracyclines TREATMENT A220 skin anthrax [Cutaneous anthrax] and taxanes TREATMENT H1511 episcleritis periodica

**If you want to go more over about entity resolution model examples, you can check this comprehensive notebooks :**

https://github.com/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/Certification_Trainings/Healthcare/24.Improved_Entity_Resolvers_in_SparkNLP_with_sBert.ipynb
https://github.com/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/Certification_Trainings/Healthcare/3.Clinical_Entity_Resolvers.ipynb

### End of Notebook